In [1]:
# get api key from your .env file
import os
import requests

from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('POLYGON_API_KEY')
ALPHA_ADVANTAGE_API_KEY = os.getenv('ALPHA_ADVANTAGE_API_KEY')

print(API_KEY)
print(ALPHA_ADVANTAGE_API_KEY)

eYFFqMh3r0gYcJ4hBsJQTxsSS8s10mZB
HJ48VNC66Z0Q01V9


<h3> Dataset One: POLYGON Daily Data </h3>

Pulled from Polygon per day, with open, high, low, and close for the entire market (total 10k+). Polygon has a API call limit of 5 / minute, so I can write a script to make five calls per minute to gather a year of data. 

<b> Potential limitations: </b> With daily data, perhaps the data will not be granular enough to provide accurate results.


In [17]:
import datetime
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def get_data(year, month, day, numDays):
    waitCounter = 0
    return_df = pd.DataFrame()
    date = datetime.date(year, month, day)
    for i in range(numDays):
        payload = {'adjusted': 'true', 'include_otc': 'false','apiKey':API_KEY}
        URL = 'https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/' + date.strftime("%Y-%m-%d")
        req = requests.get(URL, params=payload)

        if "results" in req.json():
            dat = (req.json())['results']

            dat_df = pd.DataFrame(dat)
            dat_df['date'] = date.strftime("%Y-%m-%d")
            return_df = pd.concat([return_df, dat_df], ignore_index=True)

        date += datetime.timedelta(days=1)
        if(date.weekday() == 5):
            date += datetime.timedelta(days=2)

        # 5 API call / minute limit for free account
        waitCounter += 1
        if(waitCounter >= 5):
            time.sleep(65)
            waitCounter = 0
        
    return return_df


In [21]:
# daily_dat = get_data(2023, 1, 9, 9)
# daily_dat.to_json('daily_data.json', orient = 'split', compression = 'infer', index = 'true')

df = pd.read_json('daily_data.json', orient ='split', compression = 'infer')
df

,T,v,vw,o,c,h,l,t,n,date
0,TTMI,394280.0,16.1078,15.9600,16.0800,16.3350,15.9600,1673298000000,5416.0,2023-01-09
1,OEC,485157.0,19.0627,18.6700,18.9800,19.4300,18.4500,1673298000000,8130.0,2023-01-09
2,USDU,642813.0,25.8745,25.9200,25.8800,25.9500,25.8204,1673298000000,2417.0,2023-01-09
3,BWXT,389669.0,57.4657,58.1500,57.2800,58.1800,57.0250,1673298000000,7325.0,2023-01-09
4,WRB,1298028.0,73.2759,74.2900,72.9800,74.3800,72.5900,1673298000000,15947.0,2023-01-09
...,...,...,...,...,...,...,...,...,...,...
87560,IWTR,53.0,27.1884,27.0869,27.0869,27.0869,27.0869,1674162000000,5.0,2023-01-19
87561,FEMA,8.0,25.5197,25.3800,25.3800,25.3800,25.3800,1674162000000,3.0,2023-01-19
87562,PSCM,376.0,66.9117,66.9400,66.9400,66.9400,66.9400,1674162000000,15.0,2023-01-19
87563,WGRO,0.0,NaN,20.1365,20.1365,20.1365,20.1365,1674162000000,NaN,2023-01-19


In [13]:
df[df["date"] == "2023-01-17"]

,T,v,vw,o,c,h,l,t,n,date,percent_change


In [8]:
daily_df = df
daily_df['percent_change'] = 2 * (daily_df['c'] - daily_df['o']) / (daily_df['c'] + daily_df['o'])
daily_df

,T,v,vw,o,c,h,l,t,n,date,percent_change
0,TTMI,394280.0,16.1078,15.9600,16.0800,16.3350,15.9600,1673298000000,5416.0,2023-01-10,0.007491
1,OEC,485157.0,19.0627,18.6700,18.9800,19.4300,18.4500,1673298000000,8130.0,2023-01-10,0.016467
2,USDU,642813.0,25.8745,25.9200,25.8800,25.9500,25.8204,1673298000000,2417.0,2023-01-10,-0.001544
3,BWXT,389669.0,57.4657,58.1500,57.2800,58.1800,57.0250,1673298000000,7325.0,2023-01-10,-0.015074
4,WRB,1298028.0,73.2759,74.2900,72.9800,74.3800,72.5900,1673298000000,15947.0,2023-01-10,-0.017790
...,...,...,...,...,...,...,...,...,...,...,...
65688,CLRG,32.0,32.5727,32.5600,32.5600,32.5600,32.5600,1673989200000,15.0,2023-01-18,0.000000
65689,NZUS,0.0,NaN,22.1362,22.1362,22.1362,22.1362,1673989200000,NaN,2023-01-18,0.000000
65690,BSJU,122.0,25.2080,25.2150,25.2150,25.2150,25.2150,1673989200000,5.0,2023-01-18,0.000000
65691,ARIZW,500.0,0.0110,0.0110,0.0110,0.0110,0.0110,1673989200000,1.0,2023-01-18,0.000000


<h3> Dataset Two: POLYGON Aggregates </h3>

Pulled from POLYGON per stock, minute by minute data for an individual stock. If I choose to focus on a specific market and pick a set of specific stocks, can create a script to pull all of the data. Limit of 50000 base aggregate bars (by minute) per query. Due to this, must pull data in two-month chunks (which means that we can approximately pull a year of data for a single stock in a bit over a minute).

In [4]:
payload = {'adjusted': 'true', 'sort': 'asc','apiKey':API_KEY, 'limit':'50000'}
date_start = '2022-11-01'
date_end = '2023-01-01'
stock = 'AAPL'
URL = 'https://api.polygon.io/v2/aggs/ticker/' + stock + '/range/1/minute/' + date_start + '/' + date_end 

print(URL)

minute_req = requests.get(URL, params=payload)

https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2022-11-01/2023-01-01


In [5]:
minute_json = minute_req.json()

minute_data = minute_json['results']

print(len(minute_data))

33253


<h3> Dataset Three: Alpha Advantage Fundamental Data </h3>

Pulled from 'alphadvantage' per company. If I choose to focus on a specific set of stocks, then i can pull fundamental data for a select few companies through this. This specific example is for the quarterly balance sheets for a company. I'll be able to retrieve other fundamental + stock data as well, for specific companies.

In [ ]:
advantage_payload = {'function':'BALANCE_SHEET','symbol':'AAPL','apikey':ALPHA_ADVANTAGE_API_KEY}

advantage_URL = 'https://www.alphavantage.co/query'

adv_req = requests.get(advantage_URL, params=advantage_payload)

In [ ]:
adv_json = adv_req.json()

quarterly_reports = adv_json['quarterlyReports']

print(len(quarterly_reports))